In [1]:
from utils.df_handle import *

In [2]:
query = \
    """
    SELECT DISTINCT 
    SlsperID as manv,
    CASE WHEN u.UserTypes LIKE '%LOG%' THEN 'LOG' ELSE 'MDS' END as position,
    u.FirstName as tencvbh,
     --SupID,
    sup.FirstName as tenquanlytt,
     --ASM,
    asm.FirstName as tenquanlykhuvuc,
     --RSMID,
    rsm.FirstName as tenquanlyvung,
    'DMS' as datatype
    FROM dbo.fr_ListSaleByData('admin') as a
    LEFT JOIN dbo.Users u WITH (NOLOCK) ON u.UserName=a.SlsperID
    LEFT JOIN dbo.Users sup WITH (NOLOCK) ON sup.UserName=a.SupID
    LEFT JOIN dbo.Users asm WITH (NOLOCK) ON asm.UserName=a.ASM
    LEFT JOIN dbo.Users Rsm WITH (NOLOCK) ON rsm.UserName =a.RSMID
	WHERE sup.UserName IS NOT NULL
    """
df1 = get_ms_df(query)

In [3]:
# vc(df1, 'tenquanlykhuvuc')

In [4]:
df1['tenquanlykhuvuc'].replace("Dương Tấn Bửu", "Dương Tấn Bửu(KN)", inplace=True)
df1['tenquanlykhuvuc'].replace("Nguyễn Thọ Chiến", "Nguyễn Thọ Chiến(KN)", inplace=True)

In [5]:
df_users = get_ps_df("select manv from d_users")
if df_users.shape[0] != df1.shape[0]:
    df1['tenquanlytt'].fillna(df1.tenquanlykhuvuc, inplace=True)
    df1['tenquanlykhuvuc'].fillna(df1.tenquanlyvung, inplace=True)
    # df1['tenquanlyvung'].fillna(df1.tenquanlykhuvuc, inplace=True)
    commit_psql("truncate table d_users cascade;")
    execute_values_insert(df1, "d_users")
else:
    print("None")

Data inserted using execute_values() successfully..


In [8]:
# df1[df1['tenquanlyvung'].isna()]

In [9]:
df1['tenquanlytt'].fillna(df1.tenquanlykhuvuc, inplace=True)
df1['tenquanlykhuvuc'].fillna(df1.tenquanlyvung, inplace=True)
#13/01
df1.drop_duplicates(keep='last', inplace=True, subset=['manv'])
commit_psql("truncate table d_users cascade;")
execute_values_insert(df1, "d_users")

Data inserted using execute_values() successfully..


In [8]:
df1[checkdup(df1, 2, ['manv'])].to_clipboard()